In [7]:
import numpy as np
import pandas as pd
import os
import pickle
#import settings
import plotly
import dash
import dash_core_components as dcc   
import dash_html_components as html
import plotly.graph_objs as go 
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
#選手一覧データの読み込み
df = pd.read_pickle('C:/Users/kurotsu-mayumi/Anaconda3/analysis_bb_gold/pickles_alt/player_info8.pkl')

In [3]:
df.shape

(924, 44)

In [4]:
df.columns

Index(['背番号', '選手名', '守備', '生年月日', '年齢', 'プロ年数', '身長', '体重', '血液型', '投打',
       '出身地', '推定年俸(万円)', '2019年所属球団', '生年年', '生年月', '生年日', '選手名_球団',
       '選手名_生年月日', '出場試合数', '出身地域', '出身校', '高・大フラグ', 'オリックス', 'ソフトバンク', 'ヤクルト',
       'ロッテ', '中日', '巨人', '広島', '日ハム', '楽天', '横浜', '西武', '阪神',
       '直近10年間月間MVP受賞回数', '直近10年間GG受賞回数', '最大所属年数', '最大所属年数球団', 'ネックレス太さ',
       '目的変数_ネックレス装着フラグ', '外国人フラグ', '育成', '2019年セパフラグ', '最大所属年数球団_adj'],
      dtype='object')

### コールバック無し（グラフ１つ）

In [5]:
app = dash.Dash(__name__) #インスタンスの生成

#レイアウトでアプリケーションの構造を記述する
app.layout = html.Div(
    children = [
            html.H1(children = 'Dashでプロ野球選手属性の可視化をしてみる'),
            html.Div(
                children = '''
                Dashたのしい。
                '''),
            dcc.Graph( #Graphメソッドは引数にplotlyのFigureオブジェクトをとる
                id = 'example-graph',
                figure = {
                    'data':[
                        go.Histogram(
                            x = df['生年月'],
                            marker = dict(color='#33D7E9'),
                            opacity = 0.6)],
                    'layout': go.Layout(
                        title = '生年月',
                        yaxis =dict(title='人数'),
                        bargap =0.3)
                        }
            )
    ]
)

#実行用
if __name__ == '__main__':
    app.run_server(debug = True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


### コールバック無（Subplots)

In [9]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__,external_stylesheets=external_stylesheets)

#生年月をあらかじめ集約しておく
df_bar = df.groupby('生年月', as_index = False)['2019年所属球団'].count()
df_bar = df_bar.rename(columns = {'2019年所属球団': 'count'})

#出身地域を集約しておく
df_region = df.groupby(['出身地域', '出身地'], as_index = False)['2019年所属球団'].count()
df_region = df_region.rename(columns = {'2019年所属球団': 'count'})
df_region['日本/海外'] = df_region['出身地域'].apply(lambda x: '海外' if x in ['アメリカ', 'ドミニカ共和国', 'キューバ', 'ベネズエラ', 'プエルトリコ', 
                                                          'メキシコ','カナダ', 'ブラジル', 'タイ', 'アンティル', 'オランダ', '台湾'] else '日本')
df_region['地球'] = '地球'

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("生年月", '出身地域', ))

#生年月
fig.add_trace(
    go.Bar(x=df_bar["生年月"],
           y=df_bar['count']),
    row=1, col=1
)

#出身地域
fig.add_trace(
    go.Bar(x=df_region['出身地域'],
          y=df_region['count']),
    row=1, col=2
)


app.layout = html.Div(
    children = [
        html.H2(children = 'Dashでプロ野球選手属性をインタラクティブな可視化をしてみる'),
        html.Div(
            children = '''
            コールバック無しでもいい感じ
            '''),

    dcc.Graph(id="subplotGraph", figure = fig)
])


if __name__ == '__main__':
    app.run_server(debug = True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


### コールバック例：球団ｘ選手の年齢分布

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div(
    children = [
        html.H2(children = 'Dashでプロ野球選手属性をインタラクティブな可視化をしてみる'),
        html.Div(
            children = '''
            球団を選択
            '''),
    dcc.Dropdown(
        id = 'dropdown-example',
        options = [{'label': i, 'value': i} for i in df['2019年所属球団'].unique()],
        value = df['年齢'].mean()
    ),
    dcc.Graph(
        id="AgeGraph",
    )
])

# ③コールバック作成
@app.callback(
    dash.dependencies.Output('AgeGraph', 'figure'),
    [dash.dependencies.Input('dropdown-example', 'value')]
)
def update_graph(factor):
    dff = df[df['2019年所属球団'] == factor]

    return {
        'data': [go.Histogram(
            x = dff['年齢'],
            xbins = dict(start = 18, end = 45),
            marker = dict(color='#33D7E9'),
            opacity = 0.6)],
        'layout': go.Layout(
            title = '選手の年齢分布',
            yaxis =dict(title='人数'),
            bargap =0.3)
    }

if __name__ == '__main__':
    app.run_server(debug = True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


### コールバック：年俸ｘ球団

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div(
    children = [
        html.H2(children = 'Dashでプロ野球選手属性をインタラクティブな可視化をしてみる'),
        html.Div(
            children = '''
            球団を選択
            '''),
    dcc.Dropdown(
        id = 'dropdown-example',
        options = [{'label': i, 'value': i} for i in df['2019年所属球団'].unique()],
        value = df['推定年俸(万円)'].mean()
    ),
    dcc.Graph(
        id="salaryGraph",
    )
])

# ③コールバック作成
@app.callback(
    dash.dependencies.Output('salaryGraph', 'figure'),
    [dash.dependencies.Input('dropdown-example', 'value')]
)
def update_graph(factor):
    dff = df[df['2019年所属球団'] == factor]

    return {
        'data': [go.Histogram(
            x = dff['推定年俸(万円)'],
            #xbins = dict(start = 18, end = 45),
            marker = dict(color='salmon'),
            opacity = 0.6)],
        'layout': go.Layout(
            title = '推定年俸(万円)分布',
            yaxis =dict(title='万円'),
            bargap =0.3)
    }

if __name__ == '__main__':
    app.run_server(debug = True, use_reloader=False)